In [2]:
import numpy as np
import pandas as pd
import random,os,csv
import math
import numpy as np
import scipy.io
import cv2 as cv
from matplotlib import pyplot as plt
from IPython.display import display


In [3]:
src_csv = 'C:/Users/soyou/Documents/gd_csv_out/gd_csv_out/'
src_img = 'C:/Users/soyou/Documents/gd_aa_resize_128/gd_aa_resize_128/'
dst_csv = 'C:/Users/soyou/Documents/gd_csv_out/gd_csv_out_out/'
dst_img = 'C:/Users/soyou/Documents/gd_aa_resize_128/gd_aa_resize_128_out/'
files_csv = os.listdir(src_csv)
files_img = os.listdir(src_img)
#files_gd_aa2 = os.listdir(src_gd_aa2)

## 512x512 이미지 크기 축소 (--> 128X128)
ㅇ 입력 X 로 들어갈 이미지가 128이미지이기 때문에 한번 보정한 이미지(512)를 128사이즈로 축소 시킵니다. 

In [4]:
def OpenCV_plot(src,file):
    img = cv.imread(src+file,cv.COLOR_BGR2GRAY)
    plt.imshow(img)
    plt.xticks([]) # x축 눈금
    plt.yticks([]) # y축 눈금
    plt.show()
    
    
def OpenCV_resize(src,file,width,height):
    img = cv.imread(src+file,cv.COLOR_BGR2GRAY)
    img_resize = cv.resize(img,(width,height))
    print(img_resize.shape)
    
    return img_resize

def OpenCV_write(dst,file,img):
    cv.imwrite(dst+file,img,[cv.IMWRITE_JPEG_QUALITY,100])
    print('save shape of image:',img.shape)
    print('save!:', dst+file)
    

In [5]:
img = cv.imread(src_gd_aa2+files_gd_aa2[0],cv.COLOR_BGR2GRAY)

for i in range(len(files_gd_aa2 )):
    OpenCV_plot(src_gd_aa2,files_gd_aa2[i])
    img_resize = OpenCV_resize(src_gd_aa2, files_gd_aa2[i],128,128)
    #OpenCV_write(dst, files_gd_aa2[i],img_resize)

NameError: name 'src_gd_aa2' is not defined

## CSV 데이터 정리 

ㅇ 해당내용을 리스트에 정리하는 과정입니다. 

In [6]:
files_num =[]
for i in range(len(files_csv)):
    files_num.append(int(files_csv[i].split('_')[1]))
#오름차순 정렬 
files_num_sorted =sorted(files_num)

In [7]:
temp_ = [[] for i in range(len(files_csv))]

n = 0
for i in files_num_sorted:
    file = files_csv[files_csv.index('gd_'+str(i)+'_output.csv')]
    with open(src_csv+file, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            temp_[n].append(row)
        n +=1


In [8]:
name,volume,mass = [],[],[]

for i in range(len(files_csv)):
    for j in range(3):
        if j == 0:
            name.append(temp_[i][j][1])   
        elif j == 1:
            volume.append(temp_[i][j][1])        
        elif j == 2:
            mass.append(temp_[i][j][1])

In [9]:
#특정문자 지우기 
for i in range(len(mass)):
    mass[i]=mass[i].strip('\n.')
    name[i]=name[i].strip('_output')

In [15]:
freq_list,eig_val_list,stiff_list = [[] for i in range(15)],[[] for i in range(15)],[[] for i in range(15)]

for i in range(len(temp_)):
    n = 0
    for j in range(4,19):
        freq_list[n].append(temp_[i][j][1])
        eig_val_list[n].append(temp_[i][j][2])
        stiff_list[n].append(temp_[i][j][3])
        n+=1

## Pandas 데이터 프레임 만들기 
ㅇ 데이터 프레임을 만들어 정리한 리스트를 넣는 과정입니다

In [16]:
# 키로 사용할 이름 생성하기
freq_names,eig_val_names,stiff_names = ['freq_'+str(i) for i in range(15)], ['eig_val_'+str(i) for i in range(15)], ['stiff_'+str(i) for i in range(15)]

In [23]:
def dict_gen(name_list,contents_list,count):
    return {name_list[i]:contents_list[i] for i in range(count)}

n = {"name":name}
v = {"volume":volume}
m = {"mass":mass}
f = dict_gen(freq_names,freq_list,15)
e = dict_gen(eig_val_names,eig_val_list,15)
s = dict_gen(stiff_names,stiff_list,15)

data = {**n, **v, **m,**f,**e,**s} 
#딕셔너리 사전 합치기

df = pd.DataFrame(data)
#저장
df.to_csv(dst_csv+"gd_out_out.csv", mode='w')

#df 키 확인 
print(df.columns)

#df 접근
df["freq_0"].values

Index(['name', 'volume', 'mass', 'freq_0', 'freq_1', 'freq_2', 'freq_3',
       'freq_4', 'freq_5', 'freq_6', 'freq_7', 'freq_8', 'freq_9', 'freq_10',
       'freq_11', 'freq_12', 'freq_13', 'freq_14', 'eig_val_0', 'eig_val_1',
       'eig_val_2', 'eig_val_3', 'eig_val_4', 'eig_val_5', 'eig_val_6',
       'eig_val_7', 'eig_val_8', 'eig_val_9', 'eig_val_10', 'eig_val_11',
       'eig_val_12', 'eig_val_13', 'eig_val_14', 'stiff_0', 'stiff_1',
       'stiff_2', 'stiff_3', 'stiff_4', 'stiff_5', 'stiff_6', 'stiff_7',
       'stiff_8', 'stiff_9', 'stiff_10', 'stiff_11', 'stiff_12', 'stiff_13',
       'stiff_14'],
      dtype='object')


array(['1.040090E-02', '1.017693E-02', '9.939736E-03', ...,
       '1.033475E-02', '1.038937E-02', '1.008171E-02'], dtype=object)

## 2D 이미지 라벨링하기 
ㅇ 2D 이미지 파일을 불러와서 라벨링 합니다. 이작업을 할때 파일 순서를 주의해서 작업해야합니다. (저는 index 기능을 이용해서 오름차순으로 불러와서 라벨링 할 것입니다.)

* Floating point : 소수점 2번째 자리에서 반올림

* 라벨링: gd이름_11차 모드_무게_볼륨_eigenval 

In [347]:
n = 0
for i in files_num_sorted:
    file = files_img[files_img.index('gd_'+str(i)+'.jpg')]
    file_split = file[:-4].split('.')
    
    fr = str(round(float(freq_list[10][n]),2))
    ma = str(round(float(mass[n]),2))
    vo = str(round(float(volume[n]),2))
    ei = str(round(float(eigen_val_list[10][n]),2))
    
    final_filename =file_split[0]+'_'+fr+'_'+mass[n]+'_'+vo+'_'+ei+'.jpg'
    print(final_filename)
    #os.rename(src_img+file,src_img+final_filename)
    n+=1

gd_0_746.42_1.34124E-02_4982300.0_21995310.0.jpg
gd_1_911.89_1.46768E-02_5452020.0_32827800.0.jpg
gd_2_1171.13_1.75371E-02_6514530.0_54146880.0.jpg
gd_3_901.31_1.47054E-02_5462640.0_32070830.0.jpg
gd_4_1002.98_1.55238E-02_5766630.0_39714390.0.jpg
gd_5_1005.17_1.53687E-02_5709040.0_39887540.0.jpg
gd_6_1006.78_1.59427E-02_5922240.0_40015740.0.jpg
gd_7_1056.49_1.59467E-02_5923750.0_44064650.0.jpg
gd_8_977.94_1.53703E-02_5709620.0_37756140.0.jpg
gd_9_969.9_1.51501E-02_5627840.0_37137450.0.jpg
gd_10_1091.26_1.71742E-02_6379730.0_47013160.0.jpg
gd_11_938.97_1.50724E-02_5598970.0_34807070.0.jpg
gd_12_1047.82_1.59291E-02_5917200.0_43344040.0.jpg
gd_13_813.13_1.38665E-02_5150990.0_26102170.0.jpg
gd_14_924.36_1.49442E-02_5551350.0_33732180.0.jpg
gd_15_1002.34_1.58846E-02_5900680.0_39663550.0.jpg
gd_16_1041.68_1.57922E-02_5866350.0_42838270.0.jpg
gd_17_941.96_1.44104E-02_5353050.0_35028380.0.jpg
gd_18_987.04_1.44934E-02_5383880.0_38461600.0.jpg
gd_19_995.73_1.54021E-02_5721450.0_39142120.0.jpg
gd

## 처리에 해당하지 않는 파일 이동 

ㅇ 라벨링 하기전에 처리에 해당하지 않는 파일은 이동시킬때 사용합니다.

In [343]:
import os,shutil
import numpy as np

src_files = os.listdir(src_img)
out_files = os.listdir(dst_img)

src =[]
for i in src_files:
    i_split = i[:-4].split('_')
    src.append(i_split[1])
    
# 
for i in files_num_sorted:
    src.remove(str(i))
for i in src:
    shutil.move(src_img+'gd_'+i+'.jpg',dst_img)   